In [1]:
from near_functions import query_near_testnet
import sys
import subprocess
import re

In [2]:
q = """select emitted_by_contract_account_id as contract, token_id as token_id FROM public.assets__non_fungible_token_events where token_old_owner_account_id in ('') and event_kind in ('MINT') order by emitted_at_block_timestamp desc limit 10;;"""

results = query_near_testnet(q)

## Syntax for reference

Work on methods to programatically input new CLI commands with this `subprocess.run` function

`subprocess.run(["near","view","\"nft.lamns3.testnet\"","nft_token","{\"token_id\":\"415\"}"],capture_output=True, text=True).stdout.strip('\n')` 


### Change index to view below with `index_to_query`

In [3]:
index_to_query = 3

In [4]:
contract = results[index_to_query]['emitted_by_contract_account_id']
token = results[index_to_query]['token_id']
contract, token

('dev-1658501693574-27441153030365', '502')

In [5]:
contract = r'\"{}"'.format(contract)

In [6]:
token = """{{\"token_id\":\"{}\"}}""".format(token)

In [7]:
contract, token

('\\"dev-1658501693574-27441153030365"', '{"token_id":"502"}')

In [8]:
# TODO - Slicing the contract variable is necessary to get results in some queries,
# but it sometimes is unneeded and will crop off the first letter of the contract name
# Try changing index_to_query in cell 3. 

x = subprocess.run(
    [
        "near","view", contract[1:], "nft_token", token
    ],
    capture_output=True, text=True
).stdout.strip('\n').lstrip("View call: ")

### Results:

In [9]:
x

'dev-1658501693574-27441153030365.nft_token({"token_id":"502"})\n{\n  token_id: \'502\',\n  owner_id: \'ankitraj67.testnet\',\n  metadata: {\n    title: \'INITE NFT\',\n    description: \'INITE NFT DESCRIPTION\',\n    media: \'https://inite-module-images.s3.amazonaws.com/network-0.png\',\n    media_hash: null,\n    copies: null,\n    issued_at: null,\n    price: \'1000000000000000000000000\',\n    culture: null,\n    country: null,\n    creator: \'dev-1658501693574-27441153030365\',\n    on_sale: true,\n    expires_at: null,\n    starts_at: null,\n    updated_at: null,\n    extra: null,\n    reference: null,\n    reference_hash: null\n  },\n  approved_account_ids: {}\n}'

## Data Cleaning Below

In [10]:
new = x.split("\n")

In [11]:
new = [i.strip() for i in new]
new

['dev-1658501693574-27441153030365.nft_token({"token_id":"502"})',
 '{',
 "token_id: '502',",
 "owner_id: 'ankitraj67.testnet',",
 'metadata: {',
 "title: 'INITE NFT',",
 "description: 'INITE NFT DESCRIPTION',",
 "media: 'https://inite-module-images.s3.amazonaws.com/network-0.png',",
 'media_hash: null,',
 'copies: null,',
 'issued_at: null,',
 "price: '1000000000000000000000000',",
 'culture: null,',
 'country: null,',
 "creator: 'dev-1658501693574-27441153030365',",
 'on_sale: true,',
 'expires_at: null,',
 'starts_at: null,',
 'updated_at: null,',
 'extra: null,',
 'reference: null,',
 'reference_hash: null',
 '},',
 'approved_account_ids: {}',
 '}']

In [12]:
found_media = ''

for item in new:
    if 'View call' in item:
        new.remove(item)
        continue
    if 'media:' in item:
        found_media = item
    if len(item) < 5:
        new.remove(item)
        
print(new)

if found_media and "null" not in found_media:
    url = re.search("(?P<url>https?://[^\s]+)", x).group("url")
    url = url.rstrip("',")
    print()
    print(f'{url}')

['dev-1658501693574-27441153030365.nft_token({"token_id":"502"})', "token_id: '502',", "owner_id: 'ankitraj67.testnet',", 'metadata: {', "title: 'INITE NFT',", "description: 'INITE NFT DESCRIPTION',", "media: 'https://inite-module-images.s3.amazonaws.com/network-0.png',", 'media_hash: null,', 'copies: null,', 'issued_at: null,', "price: '1000000000000000000000000',", 'culture: null,', 'country: null,', "creator: 'dev-1658501693574-27441153030365',", 'on_sale: true,', 'expires_at: null,', 'starts_at: null,', 'updated_at: null,', 'extra: null,', 'reference: null,', 'reference_hash: null', 'approved_account_ids: {}']

https://inite-module-images.s3.amazonaws.com/network-0.png


## FIN